# Time series clustering using Khiva

Clustering time series is a very important method for the analysis of time series since it allows grouping the time series according to its characteristics to proceed to the correct or desired analysis / prediction on them.

Thanks to the ability to extract characteristics from [Khiva](http://khiva-python.readthedocs.io/en/latest/), we can generate a [feature](http://khiva-python.readthedocs.io/en/latest/khiva.html#module-khiva.features) matrix to apply any of the clustering methods available and gets our desired clusters.

In [103]:
%config IPCompleter.greedy=True
%matplotlib inline

import os
import warnings

from khiva.features import *
from khiva.array import *
from khiva.library import * 
from khiva.dimensionality import *

import pandas as df

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale

import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from mpl_toolkits.mplot3d import Axes3D

plt.rcParams['figure.figsize'] = [16, 5]
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=Warning)

## Why do we want to clusterize the time series?

Actually, the clustering of time series can have a wide variety of uses and applications depending on the context in which it is used.

As an example, time series clustering can be used in order to classify them with the target of knowing what time series have got the same characteristics and that way, know which of them could be suitable to concrete forecasting methods (Moving average, exponential smoothing, box-jenkins, x-11, etc), as an example.  

## Backend
Prints the backend used. CPU, CUDA and OPENCL backends are available for Khiva.  
  
> This interactive application is being execute in **hub.mybinder** which doesn't provide a GPU and its CPU is quite limited so the features extraction is going to take some time.

In [104]:
print(get_backend())

KHIVABackend.KHIVA_BACKEND_CPU


## Features extraction

Next, a total number of 53 features are going to be extracted from 100 time series after reducing the dimensions of those ones to 1000 points in order to facilitate the computation. 

As this example is being executed in **hub.mybinder**, we don't want to execute the computation as it is going to be very time-consuming, so the next cell is already executed and the result (The feautres matrix) is already saved in binary format in: `./features_extracted.npy`

In [95]:
#path = '../../energy/data/data-enerNoc/all-data/csv'
#
#file_names = []
#features_list = []
#i = 0
#
#for filename in os.listdir(path):
#    if ".csv" in filename:
#        print(filename)
#
#        file_names.append(filename)
#        data = pd.read_csv(path + "/" + filename)
#        a = visvalingam(Array([range(len(data["value"])), data["value"].as_matrix()]), int(1000))
#        arr_tmp = a.get_col(1)
#
#        features = np.stack([abs_energy(arr_tmp).to_numpy(),
#                             absolute_sum_of_changes(arr_tmp).to_numpy(),
#                             aggregated_autocorrelation(arr_tmp, 0).to_numpy(),
#                             aggregated_autocorrelation(arr_tmp, 1).to_numpy(),
#                             aggregated_autocorrelation(arr_tmp, 2).to_numpy(),
#                             aggregated_autocorrelation(arr_tmp, 3).to_numpy(),
#                             aggregated_autocorrelation(arr_tmp, 4).to_numpy(),
#                             aggregated_autocorrelation(arr_tmp, 5).to_numpy(),
#                             approximate_entropy(arr_tmp, 4, 0.5).to_numpy(),
#                             binned_entropy(arr_tmp, 5).to_numpy(),
#                             c3(arr_tmp, True).to_numpy(),
#                             count_above_mean(arr_tmp).to_numpy(),
#                             count_below_mean(arr_tmp).to_numpy(),
#                             cwt_coefficients(arr_tmp, Array([1, 2, 3], dtype.s32), 2, 2).to_numpy(),
#                             energy_ratio_by_chunks(arr_tmp, 2, 0).to_numpy(),
#                             first_location_of_maximum(arr_tmp).to_numpy(),
#                             first_location_of_minimum(arr_tmp).to_numpy(),
#                             has_duplicates(arr_tmp).to_numpy(),
#                             has_duplicate_max(arr_tmp).to_numpy(),
#                             has_duplicate_min(arr_tmp).to_numpy(),
#                             index_mass_quantile(arr_tmp, 0.5).to_numpy(),
#                             kurtosis(arr_tmp).to_numpy(),
#                             large_standard_deviation(arr_tmp, 0.4).to_numpy(),
#                             last_location_of_maximum(arr_tmp).to_numpy(),
#                             last_location_of_minimum(arr_tmp).to_numpy(),
#                             length(arr_tmp).to_numpy(),
#                             longest_strike_above_mean(arr_tmp).to_numpy(),
#                             longest_strike_below_mean(arr_tmp).to_numpy(),
#                             max_langevin_fixed_point(arr_tmp, 7, 2).to_numpy(),
#                             maximum(arr_tmp).to_numpy(),
#                             mean(arr_tmp).to_numpy(),
#                             mean_absolute_change(arr_tmp).to_numpy(),
#                             mean_change(arr_tmp).to_numpy(),
#                             mean_second_derivative_central(arr_tmp).to_numpy(),
#                             median(arr_tmp).to_numpy(),
#                             minimum(arr_tmp).to_numpy(),
#                             number_crossing_m(arr_tmp, 0).to_numpy(),
#                             number_cwt_peaks(arr_tmp, 2).to_numpy(),
#                             number_peaks(arr_tmp, 2).to_numpy(),
#                             percentage_of_reoccurring_datapoints_to_all_datapoints(arr_tmp, False).to_numpy(),
#                             percentage_of_reoccurring_values_to_all_values(arr_tmp, False).to_numpy(),
#                             quantile(arr_tmp, Array([0.6], dtype.f32)).to_numpy(),
#                             ratio_beyond_r_sigma(arr_tmp, 0.5).to_numpy(),
#                             ratio_value_number_to_time_series_length(arr_tmp).to_numpy(),
#                             skewness(arr_tmp).to_numpy(),
#                             standard_deviation(arr_tmp).to_numpy(),
#                             sum_of_reoccurring_values(arr_tmp).to_numpy(),
#                             sum_values(arr_tmp).to_numpy(),
#                             symmetry_looking(arr_tmp, 0.1).to_numpy(),
#                             time_reversal_asymmetry_statistic(arr_tmp, 2).to_numpy(),
#                             value_count(arr_tmp, mean(arr_tmp).to_numpy()).to_numpy(),
#                             variance(arr_tmp).to_numpy(),
#                             int(variance_larger_than_standard_deviation(arr_tmp).to_numpy())])
#
#        features_list.append(features)
#np.save("features_extracted", np.array(features_list))

## Features

Next, the 53 features used in order to clusterize the time sereis are shown. As it has been explained before, the features extracted are loaded from a binary file located in `./features_extracted.npy`.

> Note: Remember to visit the [Khiva's documentation](http://khiva-python.readthedocs.io/en/latest/khiva.html#module-khiva.features) in order to understand the features extracted. 

In [96]:
pandasDF = pd.DataFrame(data=np.load("./features_extracted.npy"), columns=['abs_energy(arr)',
                                                     'absolute_sum_of_changes(arr)',
                                                     'aggregated_autocorrelation(arr, 0)',
                                                     'aggregated_autocorrelation(arr, 1)',
                                                     'aggregated_autocorrelation(arr, 2)',
                                                     'aggregated_autocorrelation(arr, 3)',
                                                     'aggregated_autocorrelation(arr, 4)',
                                                     'aggregated_autocorrelation(arr, 5)',
                                                     'approximate_entropy(arr, 4, 0.5)',
                                                     'binned_entropy(arr, 5)',
                                                     'c3(arr, True)',
                                                     'count_above_mean(arr)',
                                                     'count_below_mean(arr)',
                                                     'cwt_coefficients(arr, Array([1, 2, 3], dtype.s32), 2, 2)',
                                                     'energy_ratio_by_chunks(arr, 2, 0)',
                                                     'first_location_of_maximum(arr)',
                                                     'first_location_of_minimum(arr)',
                                                     'has_duplicates(arr)',
                                                     'has_duplicate_max(arr)',
                                                     'has_duplicate_min(arr)',
                                                     'index_mass_quantile(arr, 0.5)',
                                                     'kurtosis(arr)',
                                                     'large_standard_deviation(arr, 0.4)',
                                                     'last_location_of_maximum(arr)',
                                                     'last_location_of_minimum(arr)',
                                                     'length(arr)()',
                                                     'longest_strike_above_mean(arr)',
                                                     'longest_strike_below_mean(arr)',
                                                     'max_langevin_fixed_point(arr, 7, 2)',
                                                     'maximum(arr)',
                                                     'mean(arr)',
                                                     'mean_absolute_change(arr)',
                                                     'mean_change(arr)',
                                                     'mean_second_derivative_central(arr)',
                                                     'median(arr)',
                                                     'minimum(arr)',
                                                     'number_crossing_m(arr, 0)',
                                                     'number_cwt_peaks(arr,2 )',
                                                     'number_peaks(arr, 2)',
                                                     'percentage_of_reoccurring_datapoints_to_all_datapoints(arr, False)',
                                                     'percentage_of_reoccurring_values_to_all_values(arr, False)',
                                                     'quantile(arr, Array([0.6], dtype.f32))',
                                                     'ratio_beyond_r_sigma(arr, 0.5)',
                                                     'ratio_value_number_to_time_series_length(arr)',
                                                     'skewness(arr)',
                                                     'standard_deviation(arr)',
                                                     'sum_of_reoccurring_values(arr)',
                                                     'sum_values(arr)',
                                                     'symmetry_looking(arr, 0.1)',
                                                     'time_reversal_asymmetry_statistic(arr, 2)',
                                                     'value_count(arr, mean(arr))',
                                                     'variance(arr)',
                                                     'int(variance_larger_than_standard_deviation(arr)'])
pandasDF.head(5)

,abs_energy(arr),absolute_sum_of_changes(arr),"aggregated_autocorrelation(arr, 0)","aggregated_autocorrelation(arr, 1)","aggregated_autocorrelation(arr, 2)","aggregated_autocorrelation(arr, 3)","aggregated_autocorrelation(arr, 4)","aggregated_autocorrelation(arr, 5)","approximate_entropy(arr, 4, 0.5)","binned_entropy(arr, 5)",...,ratio_value_number_to_time_series_length(arr),skewness(arr),standard_deviation(arr),sum_of_reoccurring_values(arr),sum_values(arr),"symmetry_looking(arr, 0.1)","time_reversal_asymmetry_statistic(arr, 2)","value_count(arr, mean(arr))",variance(arr),int(variance_larger_than_standard_deviation(arr)
0,1.088200e+06,15740.005859,0.016608,-0.001612,-0.274271,4.113637,0.176880,0.031287,0.827967,1.464085,...,0.701,0.076787,11.519624,7109.590332,30911.095703,1.0,-83.594078,0.0,132.701736,1.0
1,9.527718e+05,13398.505859,-0.008338,-0.013546,-0.224557,0.455515,0.070873,0.005023,0.726336,1.130686,...,0.121,0.883937,12.635369,3860.458496,28162.408203,0.0,-28.822950,0.0,159.652557,1.0
2,7.761332e+07,27890.650391,0.089911,0.005655,-0.433963,8.225249,0.540984,0.292664,0.650519,1.273136,...,0.987,-0.583592,56.365067,3642.331543,272829.843750,1.0,-9175.507812,0.0,3177.020752,1.0
3,2.154056e+05,8028.248047,0.003225,-0.011927,-0.205272,0.499413,0.099070,0.009815,0.688051,1.147709,...,0.113,1.007509,8.879619,1835.268188,11685.797852,0.0,43.162048,0.0,78.847633,1.0
4,5.628311e+04,4691.341309,-0.005037,-0.009202,-0.259754,0.384046,0.079796,0.006367,0.751102,1.031324,...,0.115,1.239200,4.341114,830.674927,6118.645020,1.0,-6.979475,0.0,18.845266,1.0


## Clustering 

Next, a clustering based on the features extracted before is going to be carried out giving the user to select the desired number of clusters. 

Inorder to plot the results, a PCA is applied to use three principal components. 


In [102]:
# Converts the Pandas dataFrame to a numpy array.
X = pandasDF.as_matrix()
# Converts NaN to 0.
X = np.nan_to_num(X)
# Preprocess the Features matirx by scaling it.
X = scale(X)
# Applies a Principal Component Analysis with a desired number of components equal to three.
pca = PCA(n_components=3)
# Create a pandas dataframe composed by the principal components.
principal_components = pca.fit_transform(X)
principal_df = pd.DataFrame(data = principal_components
             , columns = ['principal component 1', 'principal component 2', 'principal component 3'])
# Plots the results of applying Kmeans with the desired number of clusters.
def run_prediction(clusters):
    kmeans = KMeans(n_clusters=int(clusters), random_state=0).fit(X)
    fig = plt.figure()
    Axes3D(fig).scatter(xs=principal_df['principal component 1'], ys = principal_df['principal component 2'], zs=principal_df['principal component 3'], c=kmeans.labels_, s=250)
    plt.title("Time Series Clustering - K-means( k=" + str(clusters) + ")")
    plt.show()
   
interact(run_prediction,clusters=IntSlider(min=1, max=len(X), step=1, continuous_update=False));

interactive(children=(IntSlider(value=1, continuous_update=False, description='clusters', min=1), Output()), _…